# Feature Extraction Phase
In this notebook I'm running the feature extraction phase using an Inception Convolutional Neural Network as the feature extraction model trained with the pre-processed data as well as optimized with a Bayesian Optimization process.

**Author**: Arthur G.

## Loading Dependencies
Loading and setting up all the dependencies for this notebook.

In [1]:
# libs
import os
import time
import warnings

import numpy as np
import pandas as pd
import tensorflow as tf
from pycaret.classification import *
from keras_tuner import HyperModel, Objective
from keras_tuner.tuners import BayesianOptimization
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.datasets import make_classification
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau
from tensorflow.keras.callbacks import ModelCheckpoint

# settings
seed = np.random.seed(42)
warnings.filterwarnings("ignore")

2023-01-23 16:10:23.553184: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-01-23 16:10:23.770923: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-01-23 16:10:23.830532: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-23 16:10:23.830545: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if yo

## Helper Functions
A set of helper functions and classes for the automation of model optimization.

In [2]:
class InceptionHyperModel(HyperModel):
    """Builds the base Inception model for dynamic optimization."""
    
    def build(self, hp):
        """Initiate the model before the bayesian process gets started."""
        input_tensor = tf.keras.layers.Input(shape=(1, 300, 20)) # era 1,300,1

        
        # bottleneck layer
        x = tf.keras.layers.Conv2D(
                filters=hp.Int("n_filters_1_bn", min_value=16, max_value=64, step=16), 
                kernel_size=(1, 1), 
                strides=1, 
                padding='same', 
                activation='relu'
        )(input_tensor)
        x = tf.keras.layers.Conv2D(
                filters=hp.Int("n_filters_2_bn", min_value=16, max_value=64, step=16), 
                kernel_size=hp.Int("kernel_size_bn", min_value=2, max_value=4, step=1), 
                strides=1, 
                padding='same', 
                activation='relu'
        )(x)
        x = tf.keras.layers.Conv2D(
                filters=hp.Int("n_filters_3_bn", min_value=16, max_value=64, step=16), 
                kernel_size=(1, 1), 
                strides=1, 
                padding='same', 
                activation='relu'
        )(x)
        x = tf.keras.layers.Dropout(
                hp.Choice("dropout_rate_bn", values=[0.05, 0.5], default=0.5)
        )(x)
        

        # dynamic efficient inception blocks
        for i in range(hp.Int("n_incep_blocks", min_value=2, max_value=4, step=1)):
            x = tf.keras.layers.Conv2D(
                    filters=hp.Int("n_filters_1_ib", min_value=16, max_value=64, step=16), 
                    kernel_size=(1, 1), 
                    padding="same", 
                    activation="relu"
            )(x)
            x = tf.keras.layers.Conv2D(
                    filters=hp.Int("n_filters_2_ib", min_value=16, max_value=64, step=16), 
                    kernel_size=(1, 1), 
                    padding="same", 
                    activation="relu"
            )(x)
            x = tf.keras.layers.Conv2D(
                    filters=hp.Int("n_filters_3_ib", min_value=16, max_value=64, step=16), 
                    kernel_size=(3, 3), 
                    padding="same", 
                    activation="relu"
            )(x)
            x = tf.keras.layers.Conv2D(
                    filters=hp.Int("n_filters_4_ib", min_value=16, max_value=64, step=16), 
                    kernel_size=(1, 1), 
                    padding="same", 
                    activation="relu"
            )(x)
            x = tf.keras.layers.Conv2D(
                    filters=hp.Int("n_filters_5_ib", min_value=16, max_value=64, step=16), 
                    kernel_size=(5, 5), 
                    padding="same", 
                    activation="relu"
            )(x)
            x = tf.keras.layers.MaxPooling2D(
                    pool_size=(3, 3), 
                    strides=(1, 1), 
                    padding='same'
            )(x)
            x = tf.keras.layers.Conv2D(
                filters=hp.Int("n_filters_6_ib", min_value=16, max_value=64, step=16), 
                kernel_size=(1, 1), 
                padding="same", 
                activation="relu"
            )(x)
        

        # fully connected layer
        x = tf.keras.layers.GlobalAveragePooling2D()(x)
        
        for i in range(hp.Int("n_fc_blocks", min_value=2, max_value=4, step=1)):
            x = tf.keras.layers.Dense(
                    units=hp.Int("n_dense_nodes_1_fc", min_value=32, max_value=256, step=32), 
                    activation='relu'
            )(x)
            x = tf.keras.layers.Dropout(
                    hp.Choice("dropout_rate_fc", values=[0.05, 0.5], default=0.5)
            )(x)
            x = tf.keras.layers.Dense(
                    units=hp.Int(
                        "n_dense_nodes_2_fc", min_value=32, max_value=256, step=32) \
                        // hp.Int("dense_nodes_div", min_value=2, max_value=8, step=2), 
                        activation='relu'
            )(x)
        
        output_tensor = tf.keras.layers.Dense(
            4, 
            activation='softmax'
        )(x)
        

        # model compilation
        model = tf.keras.Model(inputs=input_tensor, outputs=output_tensor)
        model.compile(
            loss='sparse_categorical_crossentropy',
            metrics=['accuracy'],
            optimizer=hp.Choice(
                "optimizer", 
                values=[
                    "sgd", "rmsprop", "adam", 
                    "adadelta", "adagrad", 
                    "adamax", "nadam"
                ], default="adam"
            )
        )
        
        return model
    

def get_features_extracted(model, final_layer: str, sample: np.ndarray):
    """
    This function extracts outputs from a given layer
    name.
    """
    sample = np.expand_dims(sample, axis=0)
    output = np.array(
        tf.keras.Model(inputs=model.inputs, outputs=model.get_layer(final_layer).output)(sample)
    )
    
    return output


## Feature Extraction Pipeline
Building the feature extraction pipeline using an Inception Convolutional Neural Network.

Loading and preparing the DCE-based dataset.

In [3]:
# loading data and targets
data = np.loadtxt(os.path.join("..","..", "data", "processed", "EEG_filt_20_electrodes.csv"), delimiter=',')
targets = np.loadtxt(os.path.join("..","..", "data", "processed", "EEG_filt_first_8_electrodes_targets.csv"), delimiter=',')

# encoding targets (from string to int)
targets[targets == 0] = 0
targets[targets == 1] = 1 # 19494 subjects of this class
targets[targets == 2] = 2
targets[targets == 3] = 3 # 11286 subjects of this class
targets = targets.astype(int)

print(f"Original data shape: {data.shape}")
data_reshape = data.reshape(10800, 1,300, 20) 

print(f"Reshaped data shape: {data_reshape.shape}")
print(f"Original targets shape: {targets.shape}")

Original data shape: (3240000, 20)
Reshaped data shape: (10800, 1, 300, 20)
Original targets shape: (10800,)


In [4]:
print(f"data: {data[3,1]}, data_reshape: {data_reshape[0,0,3,1]}")
data_reshape.shape

data: 65.4093722285802, data_reshape: 65.4093722285802


(10800, 1, 300, 20)

### Train-Test-Split
Splitting DCE-based dataset into train/holdout sets.

In [5]:
x_train, x_holdout, y_train, y_holdout = train_test_split(
    data_reshape,
    targets,
    test_size=0.1,
    shuffle=True,
    random_state=seed
)

print(f"Original data train set: {x_train.shape}")
print(f"Holdout data test set: {x_holdout.shape}")

Original data train set: (9720, 1, 300, 20)
Holdout data test set: (1080, 1, 300, 20)


### Architecture Optimization
Building the optimizable iCNN skeleton.

In [6]:
# loading the Inception skeleton
inception_hyper_model = InceptionHyperModel()

Setting up the architecture tuner.

In [7]:
# setting up the optimization process
tuner = BayesianOptimization(
    hypermodel=inception_hyper_model,
    objective="val_loss",
    num_initial_points=25,
    max_trials=15,
    directory=os.path.join("..","..", "models","updatedInception"),
    project_name="20_electrodes_4_classes_10_test_90_train_feature_extractor",
    seed=seed
)

INFO:tensorflow:Reloading Oracle from existing project ../../models/updatedInception/20_electrodes_4_classes_10_test_90_train_feature_extractor/oracle.json
INFO:tensorflow:Reloading Tuner from ../../models/updatedInception/20_electrodes_4_classes_10_test_90_train_feature_extractor/tuner0.json


2023-01-23 16:11:04.183330: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:966] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-01-23 16:11:04.183425: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-01-23 16:11:04.183475: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublas.so.11'; dlerror: libcublas.so.11: cannot open shared object file: No such file or directory
2023-01-23 16:11:04.183508: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcublasLt.so.11'; dlerror: libcublasLt.so.11: cannot open shared object file: No such file or directory
2023-01-23 16:11:04.183541: W tensorflow/stream_executor/platform/default/dso_loader.cc:6

Defining callback functions.

In [8]:
# defining callbacks
callbacks = [
    EarlyStopping(
        monitor='val_loss', 
        patience=20
    ),
    ReduceLROnPlateau(
        monitor="val_loss", 
        factor=0.1, 
        patience=10, 
        mode="auto", 
        min_delta=0.0001, 
        cooldown=0, 
        min_lr=1.0e-6
    )
]

Running the tuner.

In [9]:
# running the search
tuner.search(
    x_train,
    y_train,
    epochs=250,
    validation_split=0.2,
    batch_size=32,
    callbacks=[callbacks]
)

INFO:tensorflow:Oracle triggered exit


### Building The Model
Getting the best feature extraction model from the tuner.

In [10]:
best_feature_extractor = tuner.get_best_models(num_models=1)[0]

### Feature Extraction Process
Using the model to extract predictive patterns from the DCE-based dataset.

In [11]:
train_features = []
holdout_features = []

# extracting trian features
for sample in x_train:
    sample_features = get_features_extracted(best_feature_extractor, "global_average_pooling2d", sample)
    train_features.append(sample_features)
    
# extracting holdout features
for sample in x_holdout:
    sample_features = get_features_extracted(best_feature_extractor, "global_average_pooling2d", sample)
    holdout_features.append(sample_features)

# assembling feature dataframes
train_features_df = pd.DataFrame(np.array(train_features).reshape((len(train_features),len(train_features[0][0])))) 
train_features_df["CLASS"] = y_train

holdout_features_df = pd.DataFrame(np.array(holdout_features).reshape((len(holdout_features),len(holdout_features[0][0]))))
holdout_features_df["CLASS"] = y_holdout

print(f"Train dataset shape: {train_features_df.shape}")
print(f"Holdout dataset shape: {holdout_features_df.shape}")

Train dataset shape: (9720, 33)
Holdout dataset shape: (1080, 33)


## Machine Learning Classification Experiment
Renaming the columns to avoid errors.

In [12]:
train_features_df.columns = train_features_df.columns.astype(str)
holdout_features_df.columns = holdout_features_df.columns.astype(str)

In [13]:
eval_result = best_feature_extractor.evaluate(x_holdout, y_holdout)
print("[test loss, test accuracy]:", eval_result)

34/34 [==============================] - 1s 7ms/step - loss: 0.2822 - accuracy: 0.9009
[test loss, test accuracy]: [0.28218817710876465, 0.9009259343147278]


Setting up the ML classification pipeline.

In [14]:
classif_exp = setup(
    data=train_features_df,
    test_data=holdout_features_df,
    target="CLASS",
    fold_shuffle=True,
    preprocess = False,
    use_gpu=True
)

,Description,Value
0,session_id,8444
1,Target,CLASS
2,Target Type,Multiclass
3,Label Encoded,"0: 0, 1: 1, 2: 2, 3: 3"
4,Original Data,"(9720, 33)"
5,Missing Values,False
6,Numeric Features,30
7,Categorical Features,2
8,Transformed Train Set,"(9720, 32)"
9,Transformed Test Set,"(1080, 32)"


#### Models Comparison
Comparing different machine learning model performances for the task of classifying DCE-base features.

In [ ]:
best_model = compare_models(sort="Accuracy")

IntProgress(value=0, description='Processing: ', max=79)

,,
,,
Initiated,. . . . . . . . . . . . . . . . . .,16:13:54
Status,. . . . . . . . . . . . . . . . . .,Fitting 10 Folds
Estimator,. . . . . . . . . . . . . . . . . .,Gradient Boosting Classifier


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
rf,Random Forest Classifier,0.9447,0.9911,0.9381,0.9448,0.9446,0.9127,0.9127,0.7870
knn,K Neighbors Classifier,0.9431,0.9813,0.9363,0.9433,0.9431,0.9103,0.9104,0.0480
lr,Logistic Regression,0.9247,0.9845,0.9121,0.9249,0.9245,0.8804,0.8807,0.2130
dt,Decision Tree Classifier,0.9153,0.9353,0.9069,0.9155,0.9153,0.8666,0.8667,0.1780
svm,SVM - Linear Kernel,0.9032,0.0000,0.8873,0.9042,0.8991,0.8441,0.8482,0.0350
ridge,Ridge Classifier,0.8564,0.0000,0.8177,0.8591,0.8471,0.7654,0.7744,0.0250
nb,Naive Bayes,0.8427,0.9595,0.8448,0.8528,0.8379,0.7554,0.7616,0.0180
ada,Ada Boost Classifier,0.8378,0.9100,0.8304,0.8446,0.8248,0.7412,0.7506,1.2330
qda,Quadratic Discriminant Analysis,0.5441,0.8961,0.6858,0.7532,0.5210,0.3998,0.4517,0.0340


In [ ]:
holdout_preds = predict_model(best_model)

## Models Serialization
Serializing feature extraction as well as ml classification models.

In [ ]:
# best feature extraction icnn model
best_feature_extractor.save(
    os.path.join("..","..", "models", "updatedInception", "20_electrodes_4_classes_10_test_90_train_best_feature_extractor"),
    overwrite=True,
    include_optimizer=True,
    save_format="h5",
)

## Data Serialization
Serializing the processed train and holdout validation sets.

In [ ]:
train_features_df.to_csv(os.path.join("..","..", "data", "finalized","updatedInception", "20_electrodes_4_classes_10_test_90_train_train_df.csv"), index=False)
holdout_features_df.to_csv(os.path.join("..","..", "data", "finalized","updatedInception", "20_electrodes_4_classes_10_test_90_train_holdout_df.csv"), index=False)

## Ploting model

ploting the confusion_matrix:

In [ ]:
plot_model(
    best_model, 
    plot = 'confusion_matrix'
)

ploting the learning curve:

In [ ]:
plot_model(
    best_model, 
    plot = 'auc', 
    use_train_data = False
)